In [4]:
import pandas as pd
from scipy.spatial import cKDTree

In [75]:
blind = pd.read_csv("../Data/test_blind.csv", sep=";") #fichier csv qui sera merge pas les prédiction et soumis sur Kaggle
presences = pd.read_excel("../Data/Presences_Absences_train.xlsx") #fichier csv presences/absences

In [76]:
def k_voisins(k, data, test):
    """
    données:
        k: nombre de voisins considérés 
        data: dataFrame contenant les patch_ID et les espèces associées du train-PA
        test: dataFrame provenant du CSV Kaggle contenant les informations sur les lieux où la prédiction doit être faite
    fonction: 
        Liste les espèces présentes dans les k-voisins les plus proches de chaque coordonnées du dataFrame test
        à partir des information de data. 
    résultat:
        resultat: dataFrame test auquel est ajouté la colonne "Prediction" contenant les espèces présentes dans les k-voisins
    """

    #On récupère la liste des patchID différent dans data
    dfPatchID_unique = data.drop_duplicates(subset='patchID', inplace=False)
    
    # Créez un arbre KD à partir de data pour des recherches rapides.
    tree = cKDTree(dfPatchID_unique[['lon', 'lat']])

    # Liste pour stocker les k paires de coordonnées les plus proches différentes.
    nearest_coordinates = []
    
    # Pour chaque lignes dans test, trouvez les k points les plus proches dans data.
    for patchID, row in test.iterrows():
        lon, lat = row['lon'], row['lat']
        # Recherche des k points les plus proches et récupération des indices.
        distances, indices = tree.query([lon, lat], k)

        # Ajoutez les paires de coordonnées uniques à la liste nearest_coordinates.
        nearest_coordinates.append([(dfPatchID_unique.loc[i, 'lon'], dfPatchID_unique.loc[i, 'lat']) for i in indices])

    # Ajoutez la liste des paires de coordonnées au DataFrame test.
    test['nearest_coordinates'] = nearest_coordinates

    return test

In [141]:
def k_patchID(k, data, test):
    """
    données:
        k: nombre de voisins considérés 
        data: dataFrame contenant les patch_ID et les espèces associées du train-PA
        test: dataFrame provenant du CSV Kaggle contenant les informations sur les lieux où la prédiction doit être faite
    fonction: 
        Liste les k patchID les plus proches de chaque points (lignes) du dataFrame test. 
    résultat:
        resultat: liste de liste, où la ligne i correspond aux k patchID les plus proches
        de la ligne i du dataFrame test. 
    """
    #On récupère la liste des patchID différent dans data
    dfPatchID_unique = data.drop_duplicates(subset='patchID', inplace=False)

    # Créez un arbre KD à partir de data pour des recherches rapides.
    tree = cKDTree(dfPatchID_unique[['lon', 'lat']])

    # Liste pour stocker les k paires de coordonnées les plus proches différentes.
    nearest_coordinates = []

        # Pour chaque lignes dans test, trouvez les k points les plus proches dans data.
    for patchID, row in test.iterrows():
        lon, lat = row['lon'], row['lat']
        # Recherche des k points les plus proches et récupération des indices.
        distances, indices = tree.query([lon, lat], k=k)
        nearest_coordinates.append([data.loc[i, 'patchID'] for i in indices])

    return nearest_coordinates

In [150]:
patchs = k_patchID(10, presences, blind)

In [173]:
filtered_data = presences.loc[presences['patchID'].isin(patchs[500]), ['lon', 'lat']]
filtered_data.drop_duplicates(subset='lat', inplace=True)
filtered_data

,lon,lat
131,1.776220,42.98183
227,6.688130,47.41629
341,6.210910,48.15360
656,-2.667205,52.53986
1315,2.763610,47.46125
1894,6.577550,43.69112
2168,-0.317620,50.84351
3006,6.502310,43.19040
3460,7.116885,43.83064
3526,6.404860,44.39995


In [175]:
import folium

# Créez une carte centrée sur une latitude et une longitude de départ.
m = folium.Map(location=[47, 5], zoom_start=4)

# Coordonnées (latitude, longitude) à partir de votre DataFrame "filtered_data".
for idx, row in filtered_data.iterrows():
    folium.Marker(location=[row['lat'], row['lon']]).add_to(m)

# Affichez la carte
m
